<a href="https://colab.research.google.com/github/Nathan-Ketterlinus/TianInternship/blob/SimplifiedBardina/SB_mix_Lorenz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import deepxde as dde
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.layers import TFSMLayer
import os
from functools import partial


#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# -----------------------------------
# These are the values you need to change
# -----------------------------------
alpha = 0.5
rho = 1.0
mu = 1.0
# -----------------------------------
eta_diffarray = []
eta_array = []
for eta in [30]:
#time variable for linespace
num_reference_solutions = 1000  # N_s
total_simulation_time = 110  # Simulate up to t=110
observation_start_time = 100  # Start recording observations at t=100
observations_per_time_unit = 10  # δ = 0.1 time units between observations
delta_t = 1.0/observations_per_time_unit  # 0.1

# Create observation times (from 100 to 110, every 0.1 units)
num_observations = int((total_simulation_time - observation_start_time) / delta_t) + 1
observation_times = np.linspace(observation_start_time, total_simulation_time, num_observations)

    # Full simulation times (from 0 to 110 - needed for the ODE solver)
full_simulation_times = np.linspace(0, total_simulation_time,
                                    int(total_simulation_time * observations_per_time_unit) + 1)


class Simplified_Bardina:
    def __init__(self, alpha = 0.5, mu = 1):
        self.alpa = alpha
        self.eta = eta
        self.mu = mu

    def original_system(self, x, y):
        f_x = 0
        f_y = 0
        f_z = 0


        u, v, w, u_bar, v_bar, w_bar = (y[:, 0:1], y[:, 1:2], y[:, 2:3], y[:, 3:4], y[:, 4:5], y[:, 5:6])


        # First-order derivatives for SB
        du_t = dde.grad.jacobian(y, x, i=0, j=3)  # ∂u/∂t
        du_x = dde.grad.jacobian(y, x, i=0, j=0)  # ∂u/∂x
        du_y = dde.grad.jacobian(y, x, i=0, j=1)  # ∂u/∂y
        du_z = dde.grad.jacobian(y, x, i=0, j=2)  # ∂u/∂z

        dv_t = dde.grad.jacobian(y, x, i=1, j=3)  # ∂v/∂t
        dv_x = dde.grad.jacobian(y, x, i=1, j=0)  # ∂v/∂x
        dv_y = dde.grad.jacobian(y, x, i=1, j=1)  # ∂v/∂y
        dv_z = dde.grad.jacobian(y, x, i=1, j=2)  # ∂v/∂z

        dw_t = dde.grad.jacobian(y, x, i=2, j=3)  # ∂w/∂t
        dw_x = dde.grad.jacobian(y, x, i=2, j=0)  # ∂w/∂x
        dw_y = dde.grad.jacobian(y, x, i=2, j=1)  # ∂w/∂y
        dw_z = dde.grad.jacobian(y, x, i=2, j=2)  # ∂w/∂z


        du_bar_t = dde.grad.jacobian(y, x, i=3, j=3)  # ∂u_bar/∂t
        du_bar_x = dde.grad.jacobian(y, x, i=3, j=0)  # ∂u_bar/∂x
        du_bar_y = dde.grad.jacobian(y, x, i=3, j=1)  # ∂u_bar/∂y
        du_bar_z = dde.grad.jacobian(y, x, i=3, j=2)  # ∂u_bar/∂z

        dv_bar_t = dde.grad.jacobian(y, x, i=4, j=3)  # ∂v_bar/∂t
        dv_bar_x = dde.grad.jacobian(y, x, i=4, j=0)  # ∂v_bar/∂x
        dv_bar_y = dde.grad.jacobian(y, x, i=4, j=1)  # ∂dv_bar/∂y
        dv_bar_z = dde.grad.jacobian(y, x, i=4, j=2)  # ∂dv_bar/∂z

        dw_bar_t = dde.grad.jacobian(y, x, i=5, j=3)  # ∂w_bar/∂t
        dw_bar_x = dde.grad.jacobian(y, x, i=5, j=0)  # ∂dw_bar/∂x
        dw_bar_y = dde.grad.jacobian(y, x, i=5, j=1)  # ∂dw_bar/∂y
        dw_bar_z = dde.grad.jacobian(y, x, i=5, j=2)  # ∂dw_bar/∂z

        dq_x = dde.grad.jacobian(y, x, i=6, j=0)  # ∂q/∂x
        dq_y = dde.grad.jacobian(y, x, i=6, j=1)  # ∂q/∂y
        dq_z = dde.grad.jacobian(y, x, i=6, j=2)  # ∂q/∂z

        # Second-order derivatives
      # Hessian takes:
        # y: output tensor [u, v, w, u_bar, v_bar, w_bar, q]
        # x: input tensor [x, y, z, t]
        # component: index of y[] to differentiate (y[component])
        # i: index of x to differentiate w.r.t 1st (x[i])
        # j: index of x to differentiate w.r.t 2nd (x[j])
      # and returns:
        # (∂^2 y[component]) / (∂x[i] ∂x[j])

        du_xx = dde.grad.hessian(y, x, component=0, i=0, j=0)
        du_yy = dde.grad.hessian(y, x, component=0, i=1, j=1)
        du_zz = dde.grad.hessian(y, x, component=0, i=2, j=2)

        dv_xx = dde.grad.hessian(y, x, component=1, i=0, j=0)
        dv_yy = dde.grad.hessian(y, x, component=1, i=1, j=1)
        dv_zz = dde.grad.hessian(y, x, component=1, i=2, j=2)

        dw_xx = dde.grad.hessian(y, x, component=2, i=0, j=0)
        dw_yy = dde.grad.hessian(y, x, component=2, i=1, j=1)
        dw_zz = dde.grad.hessian(y, x, component=2, i=2, j=2)

        du_bar_xx = dde.grad.hessian(y, x, component=3, i=0, j=0)
        du_bar_yy = dde.grad.hessian(y, x, component=3, i=1, j=1)
        du_bar_zz = dde.grad.hessian(y, x, component=3, i=2, j=2)

        dv_bar_xx = dde.grad.hessian(y, x, component=4, i=0, j=0)
        dv_bar_yy = dde.grad.hessian(y, x, component=4, i=1, j=1)
        dv_bar_zz = dde.grad.hessian(y, x, component=4, i=2, j=2)

        dw_bar_xx = dde.grad.hessian(y, x, component=5, i=0, j=0)
        dw_bar_yy = dde.grad.hessian(y, x, component=5, i=1, j=1)
        dw_bar_zz = dde.grad.hessian(y, x, component=5, i=2, j=2)

        # Laplacians
        laplacian_u = du_xx + du_yy + du_zz
        laplacian_v = dv_xx + dv_yy + dv_zz
        laplacian_w = dw_xx + dw_yy + dw_zz

        laplacian_u_bar = du_bar_xx + du_bar_yy + du_bar_zz
        laplacian_v_bar = dv_bar_xx + dv_bar_yy + dv_bar_zz
        laplacian_w_bar = dw_bar_xx + dw_bar_yy + dw_bar_zz

        # Relationship between filtered and actual velocities
        relation_u = u - self.alpha**2 * laplacian_u - u_bar
        relation_v = v - self.alpha**2 * laplacian_v - v_bar
        relation_w = w - self.alpha**2 * laplacian_w - w_bar


         # Momentum equations
        f1 = (
            du_bar_t                                                    # ∂v/∂t (filtered velocity)
            + (u * du_x + v * du_y + w * du_z)                          # x-component of (u · ∇)u
            + dq_x                                                      # pressure gradient (∂p/∂x)
            - self.mu * laplacian_u_bar                                      # viscosity term on u: -mu Δv
            - f_x                                                       # external force in x-direction
        )
        f2 = (
            dv_bar_t
            + (u * dv_x + v * dv_y + w * dv_z)
            + dq_y
            - self.mu * laplacian_v_bar
            - f_y
        )
        f3 = (
            dw_bar_t
            + (u * dw_x + v * dw_y + w * dw_z)
            + dq_z
            - self.mu * laplacian_w_bar
            - f_z
        )

        incompressibility = du_x + dv_y + dw_z

        return [f1, f2, f3, incompressibility]

    def nudged_system(self, x, a):

        f_x_bar = 0
        f_y_bar = 0
        f_z_bar = 0

        l, m, n, l_bar, m_bar, n_bar, q_bar = (a[:, 0:1], a[:, 1:2], a[:, 2:3], a[:, 3:4], a[:, 4:5], a[:, 5:6])

        # Get the observed x value at time t (needs interpolation)
        u = np.interp(t, y[:, 0], y[:, 1])

        dl_t = dde.grad.jacobian(a, x, i=0, j=3)  # ∂i/∂t
        dl_x = dde.grad.jacobian(a, x, i=0, j=0)  # ∂i/∂x
        dl_y = dde.grad.jacobian(a, x, i=0, j=1)  # ∂i/∂y
        dl_z = dde.grad.jacobian(a, x, i=0, j=2)  # ∂i/∂z

        dm_t = dde.grad.jacobian(a, x, i=1, j=3)  # ∂j/∂t
        dm_x = dde.grad.jacobian(a, x, i=1, j=0)  # ∂j/∂x
        dm_y = dde.grad.jacobian(a, x, i=1, j=1)  # ∂j/∂y
        dm_z = dde.grad.jacobian(a, x, i=1, j=2)  # ∂j/∂z

        dn_t = dde.grad.jacobian(a, x, i=2, j=3)  # ∂k/∂t
        dn_x = dde.grad.jacobian(a, x, i=2, j=0)  # ∂k/∂x
        dn_y = dde.grad.jacobian(a, x, i=2, j=1)  # ∂k/∂y
        dn_z = dde.grad.jacobian(a, x, i=2, j=2)  # ∂k/∂z


        dl_bar_t = dde.grad.jacobian(a, x, i=3, j=3)  # ∂l_bar/∂t
        dl_bar_x = dde.grad.jacobian(a, x, i=3, j=0)  # ∂l_bar/∂x
        dl_bar_y = dde.grad.jacobian(a, x, i=3, j=1)  # ∂l_bar/∂y
        dl_bar_z = dde.grad.jacobian(a, x, i=3, j=2)  # ∂l_bar/∂z

        dm_bar_t = dde.grad.jacobian(a, x, i=4, j=3)  # ∂m_bar/∂t
        dm_bar_x = dde.grad.jacobian(a, x, i=4, j=0)  # ∂m_bar/∂x
        dm_bar_y = dde.grad.jacobian(a, x, i=4, j=1)  # ∂m_bar/∂y
        dm_bar_z = dde.grad.jacobian(a, x, i=4, j=2)  # ∂m_bar/∂z

        dn_bar_t = dde.grad.jacobian(a, x, i=5, j=3)  # ∂n_bar/∂t
        dn_bar_x = dde.grad.jacobian(a, x, i=5, j=0)  # ∂n_bar/∂x
        dn_bar_y = dde.grad.jacobian(a, x, i=5, j=1)  # ∂n_bar/∂y
        dn_bar_z = dde.grad.jacobian(a, x, i=5, j=2)  # ∂n_bar/∂z

        dq_bar_x = dde.grad.jacobian(a, x, i=6, j=0)  # ∂q_bar/∂x
        dq_bar_y = dde.grad.jacobian(a, x, i=6, j=1)  # ∂q_bar/∂y
        dq_bar_z = dde.grad.jacobian(a, x, i=6, j=2)  # ∂q_bar/∂z

        # Second-order derivatives

        dl_xx = dde.grad.hessian(a, x, component=0, i=0, j=0)
        dl_yy = dde.grad.hessian(a, x, component=0, i=1, j=1)
        dl_zz = dde.grad.hessian(a, x, component=0, i=2, j=2)

        dm_xx = dde.grad.hessian(a, x, component=1, i=0, j=0)
        dm_yy = dde.grad.hessian(a, x, component=1, i=1, j=1)
        dm_zz = dde.grad.hessian(a, x, component=1, i=2, j=2)

        dn_xx = dde.grad.hessian(a, x, component=2, i=0, j=0)
        dn_yy = dde.grad.hessian(a, x, component=2, i=1, j=1)
        dn_zz = dde.grad.hessian(a, x, component=2, i=2, j=2)

        dl_bar_xx = dde.grad.hessian(a, x, component=3, i=0, j=0)
        dl_bar_yy = dde.grad.hessian(a, x, component=3, i=1, j=1)
        dl_bar_zz = dde.grad.hessian(a, x, component=3, i=2, j=2)

        dm_bar_xx = dde.grad.hessian(a, x, component=4, i=0, j=0)
        dm_bar_yy = dde.grad.hessian(a, x, component=4, i=1, j=1)
        dm_bar_zz = dde.grad.hessian(a, x, component=4, i=2, j=2)

        dn_bar_xx = dde.grad.hessian(a, x, component=5, i=0, j=0)
        dn_bar_yy = dde.grad.hessian(a, x, component=5, i=1, j=1)
        dn_bar_zz = dde.grad.hessian(a, x, component=5, i=2, j=2)

        # Laplacians
        laplacian_l = dl_xx + dl_yy + dl_zz
        laplacian_m = dm_xx + dm_yy + dm_zz
        laplacian_n = dn_xx + dn_yy + dn_zz

        laplacian_l_bar = dl_bar_xx + dl_bar_yy + dl_bar_zz
        laplacian_m_bar = dm_bar_xx + dm_bar_yy + dm_bar_zz
        laplacian_n_bar = dn_bar_xx + dn_bar_yy + dn_bar_zz

        # Relationship between filtered and actual velocities
        relation_l = l - alpha**2 * laplacian_l - l_bar
        relation_m = m - alpha**2 * laplacian_m - m_bar
        relation_n = n - alpha**2 * laplacian_n - n_bar

         f4 = (
            dl_bar_t                                                    # ∂i/∂t (filtered velocity)
            + (l * dl_x + m * dl_y + n * dl_z)                          # x-component of (i · ∇)i
            + dq_bar_x                                                  # pressure gradient (∂p_bar/∂x)
            - mu * laplacian_l_bar                                      # viscosity term on u: -mu Δl_bar
            - f_x_bar                                                   # external force in x-direction
            - eta * (l - u)                                             # nudging parameter
        )
        f5 = (
            dm_bar_t
            + (l * dm_x + m * dm_y + n * dm_z)
            + dq_bar_y
            - mu * laplacian_m_bar
            - f_y_bar
        )
        f6 = (
            dn_bar_t
            + (l * dn_x + m * dn_y + n * dn_z)
            + dq_bar_z
            - mu * laplacian_n_bar
            - f_z_bar
        )

        incompressibility_bar = dl_x + dm_y + dn_z


        return [f4, f5, f6, incompressibility_bar]
#Make Training Data
def simulate_SB_with_nudging():
    # Parameters
    alpha = 10.0
    eta = 30 # Nudging Parameter
    mu = 1

    num_reference_solutions = 1000  # N_s
    total_simulation_time = 110  # Simulate up to t=110
    observation_start_time = 100  # Start recording observations at t=100
    observations_per_time_unit = 10  # δ = 0.1 time units between observations
    delta_t = 1.0/observations_per_time_unit  # 0.1

    # Create observation times (from 100 to 110, every 0.1 units)
    num_observations = int((total_simulation_time - observation_start_time) / delta_t) + 1
    observation_times = np.linspace(observation_start_time, total_simulation_time, num_observations)

    # Verify observation times
    print(f"Observation times from t={observation_times[0]} to {observation_times[-1]} with {len(observation_times)} points")
    print(f"Time between observations: {observation_times[1] - observation_times[0]} (should be {delta_t})")

    # Full simulation times (from 0 to 110 - needed for the ODE solver)
    full_simulation_times = np.linspace(0, total_simulation_time,
                                       int(total_simulation_time * observations_per_time_unit) + 1)

    input_samples = [] # stores (x_bar, y_bar, z_bar, observed x) @ time t
    output_samples = [] # stores (x_bar, y_bar, z_bar) @ time t+1

    for i in range(num_reference_solutions):
        # Display progress of generating solutions
        if i % 100 == 0:
            print(f"Generating reference solution for initial condition {i}/{num_reference_solutions}")

        # Generate random initial conditions from N(0, 10)
        x0, y0, z0 = np.random.normal(loc=0, scale=10, size=3)

        # Simulate true system from t=0 to t=110
        SB = Simplified_Bardina(alpha, eta, mu)
        true_solution = odeint(SB.original_system, [x0, y0, z0], full_simulation_times)

        # Extract only the observation period (t=100 to t=110)
        obs_indices = (full_simulation_times >= observation_start_time)
        observed_times = full_simulation_times[obs_indices]
        observed_states = true_solution[obs_indices]

        # Verify we have correct number of observation points
        assert len(observed_times) == num_observations, \
               f"Expected {num_observations} observations, got {len(observed_times)}"

        # Create observations (x-component only) during observation period
        x_observed = np.column_stack((observed_times, observed_states[:, 0]))

        # Simulate nudged system during observation period only
        nudged_solution = odeint(lorenz.nudged_system, [0, 0, 0], observed_times, args=(x_observed, mu))

        # Create training pairs (Algorithm 3.1)
        for k in range(len(observed_times) - 1):
            # Input: [w(t_k), I_M(u(t_k))]
            current_state = nudged_solution[k]
            current_obs = x_observed[k, 1]  # x(t_k)
            input_pair = np.concatenate([current_state, [current_obs]])

            # Output: w(t_{k+1})
            next_state = nudged_solution[k + 1]

            input_samples.append(input_pair)
            output_samples.append(next_state)

    # Convert to numpy arrays
    input_data = np.array(input_samples)
    output_data = np.array(output_samples)

    # Plot results
     # Extract the portion of true_solution corresponding to observation_times
    true_solution_obs_period = true_solution[obs_indices]

    # Plot x components
    plt.subplot(3, 1, 1)
    plt.plot(observation_times, true_solution_obs_period[:, 0], 'b-', label='Original x')
    plt.plot(observation_times, nudged_solution[:, 0], 'r--', label='Nudged x')
    plt.ylabel('x')
    plt.legend()

    # Plot y components
    plt.subplot(3, 1, 2)
    plt.plot(observation_times, true_solution_obs_period[:, 1], 'b-', label='Original y')
    plt.plot(observation_times, nudged_solution[:, 1], 'r--', label='Nudged y')
    plt.ylabel('y')
    plt.legend()

    # Plot z components
    plt.subplot(3, 1, 3)
    plt.plot(observation_times, true_solution_obs_period[:, 2], 'b-', label='Original z')
    plt.plot(observation_times, nudged_solution[:, 2], 'r--', label='Nudged z')
    plt.ylabel('z')
    plt.xlabel('Time')
    plt.legend()

    plt.suptitle('Lorenz 63 System: Original vs Nudged Solutions (last trial)')
    plt.show()

    # Extract the portion of true_solution corresponding to observation_times
    true_solution_obs_period = true_solution[obs_indices]

    # Verification
    print("\n=== Verification ===")
    print(f"Number of initial conditions (Ns): {num_reference_solutions}")
    print(f"Observations per simulation: {num_observations - 1} (from {num_observations} time points)")
    expected_total_samples = num_reference_solutions * (num_observations - 1)
    print(f"Expected total samples: {expected_total_samples} (Ns × {num_observations - 1})")
    print(f"Actual input_data shape: {input_data.shape} (should be ({expected_total_samples}, 4))")
    print(f"Actual output_data shape: {output_data.shape} (should be ({expected_total_samples}, 3))")

    assert input_data.shape == (expected_total_samples, 4), "Input data shape mismatch"
    assert output_data.shape == (expected_total_samples, 3), "Output data shape mismatch"

    return input_data, output_data

# Run the simulation
input, nudged = simulate_SB_with_nudging()

# crop to just the first N x N_s samples
N = 15
num_reference_solutions = 1000 # Define N_s here
input = input[:num_reference_solutions * N]
nudged = nudged[:num_reference_solutions * N]



############################################################
# Algorithm 3.2: Online Usage of DNN for Data Assimilation #
############################################################
# 1: Initialize wDNN (t0) = 0 (or arbitrary)               #
# 2: Observe IM (ui(t0))                                   #
# 3: Set k = 0                                             #
# 4: while Observations are available do                   #
# 5:      wDNN (tk+1) = DNN (wDNN (tk), IM (ui(tk))        #
# 6:      Observe IM (ui(tk+1))                            #
# 7:      k = k + 1                                        #
# 8: end while                                             #
############################################################

#define Lorenz 63 Model
X_train, X_test, y_train, y_test = train_test_split(input, nudged, train_size=0.8)


data = dde.data.Triple(X_train, y_train, X_test , y_test = y_test)
net = dde.nn.FNN([4] + [50] * 3 + [3], "tanh", "Glorot normal")

model_SB = dde.Model(data, net)
model_SB.compile("adam", lr=0.001, metrics = ["l2 relative error"])
checkpointer = dde.callbacks.ModelCheckpoint("./codes/model", verbose=1, save_better_only= True)
loss_history, train_state = model_SB.train(epochs=2000, callbacks=[checkpointer])
model_SB.restore("./codes/model-"+ str(train_state.best_step) + ".ckpt", verbose=1)
#loaded_model = model_SB.restore("./codes/model-"+ str(train_state.best_step) + ".ckpt", verbose=1)


#################################################################################
#Create test data for graphs
def test_lorenz_with_nudging():
    # Parameters
    alpha = 10.0
    rho = 1.0
    beta = 8/3
    mu = 5.0  # Nudging parameter

    input_test_samples = [] # stores <num_reference_solutions> many (x_bar, y_bar, z_bar, observed x) @ time t
    output_test_samples = [] # stores <num_reference_solutions> many (x_bar, y_bar, z_bar) @ time t+1
    recursive_predictions = []

    for i in range(1):
        # Display progress of generating solutions
        if i % 100 == 0:
            print(f"Generating reference solution {i}/{1}")

        # Generate random initial conditions
        x0, y0, z0 = np.random.normal(loc=0, scale=50, size=3)

        # Simulate true system
        lorenz = Lorenz63(alpha, rho, mu)
        true_solution = odeint(lorenz.original_system, [x0, y0, z0], observation_times)

        # Create observations (x-component only)
        x_observed = np.column_stack((observation_times, true_solution[:, 0])) # I_M(u^i(t_k))

        # Simulate nudged system
        nudged_solution = odeint(SB.nudged_system, [0, 0, 0], observation_times, args=(x_observed, mu)) # W^i(t_k)

        # Create training pairs (Algorithm 3.1)
        for k in range(len(observation_times) -  1):
            # Input: [w(t_k), I_M(u(t_k))]
            current_state = nudged_solution[k]
            current_obs = x_observed[k, 1]  # x(t_k)
            input_pair = np.concatenate([current_state, [current_obs]])

            # Output: w(t_{k+1})
            next_state = nudged_solution[k + 1]

            input_test_samples.append(input_pair)
            output_test_samples.append(next_state)

        # Create training pairs (Algorithm 3.2)
        for k in range(len(observation_times) - 1): #-2 to make it the same length as others


            current_state = nudged_solution[k]
            current_obs = x_observed[k, 1]  # x(t_k)
            current_input = np.concatenate([current_state, [current_obs]])

    # Reshape the input to match the model's expected input shape (batch_size, input_dim)
            current_input = current_input.reshape(1, -1)

    # Predict the next state: w(t_{k+1})
            next_state_prediction = model_SB.predict(current_input)

    # Extract the predicted state (remove the batch dimension)
            next_state_prediction = next_state_prediction[0]

    # Store the prediction
            recursive_predictions.append(next_state_prediction)

    # Update the current state for the next iteration
            current_state = next_state_prediction


    # Convert to numpy arrays
    input_test_data = np.array(input_test_samples)
    output_test_data = np.array(output_test_samples)
    recursive_pred_data = np.array(recursive_predictions)

    return input_test_data, output_test_data, recursive_pred_data

# Run the test simulation for graphs
original_test, nudged_test, recursive_test = test_lorenz_with_nudging()

###############################################################################



# Extract the components for plotting
x_recursive = recursive_test[:, 0]
y_recursive = recursive_test[:, 1]
z_recursive = recursive_test[:, 2]
t = observation_times[:len(original_test)]

# Plotting the results
plt.figure(figsize=(12, 8))

# Plot x components
plt.subplot(3, 1, 1)
plt.plot(t, original_test[:, 0], 'b-', label='Original x')
plt.plot(t, nudged_test[:, 0], 'r-', label = 'Nudging x')
plt.plot(t, x_recursive, 'm--', label='Estimated x')
plt.ylabel('x')
plt.legend()

# Plot y components
plt.subplot(3, 1, 2)
plt.plot(t, original_test[:, 1], 'b-', label='Original y')
plt.plot(t, nudged_test[:, 1], 'r-', label = 'Nudging y')
plt.plot(t, y_recursive, 'm--', label='Estimated y')
plt.ylabel('y')
plt.legend()

# Plot z components
plt.subplot(3, 1, 3)
plt.plot(t, original_test[:, 2], 'b-', label='Original z')
plt.plot(t, nudged_test[:, 2], 'r-', label = 'Nudging z')
plt.plot(t, z_recursive, 'm--', label='Estimated z')
plt.ylabel('z')
plt.xlabel('Time')
plt.legend()

plt.suptitle('Simplified Bardina System: Original vs Estimated Solutions')
plt.show()


